# CalCOFI
### Over 60 years of oceanographic data
#### Dataset: https://www.kaggle.com/datasets/sohier/calcofi
#### Table Info: https://calcofi.org/data/oceanographic-data/bottle-database/

## Questions

Can we predict chlorophyll abundance without having to measure it?

If so, which chemicals have the highest influence on the predictions?

## EDA and Data Preparation

In [1]:
from pyspark.sql.functions import *

#### Load Data

In [2]:
# Loading in spark
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAll([
    ('spark.master', 'local[2]'), 
    ('spark.app.name', 'App Name')])
    
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-25 04:05:04,303 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.2.1'

In [3]:
# Read the data
bottle = spark.read.csv("hdfs:///bottle.csv", header=True, inferSchema=True).cache()
maxRows = bottle.count()
print("There are", maxRows, "rows in the initial dataframe")

2022-05-25 04:05:18,185 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


There are 864863 rows in the initial dataframe


In [4]:
# View the inferred schema
print("There are", len(bottle.columns), "columns in the initial dataframe")
bottle.printSchema()

There are 74 columns in the initial dataframe
root
 |-- Cst_Cnt: integer (nullable = true)
 |-- Btl_Cnt: integer (nullable = true)
 |-- Sta_ID: string (nullable = true)
 |-- Depth_ID: string (nullable = true)
 |-- Depthm: integer (nullable = true)
 |-- T_degC: double (nullable = true)
 |-- Salnty: double (nullable = true)
 |-- O2ml_L: double (nullable = true)
 |-- STheta: double (nullable = true)
 |-- O2Sat: double (nullable = true)
 |-- Oxy_µmol/Kg: double (nullable = true)
 |-- BtlNum: integer (nullable = true)
 |-- RecInd: integer (nullable = true)
 |-- T_prec: integer (nullable = true)
 |-- T_qual: integer (nullable = true)
 |-- S_prec: integer (nullable = true)
 |-- S_qual: integer (nullable = true)
 |-- P_qual: integer (nullable = true)
 |-- O_qual: integer (nullable = true)
 |-- SThtaq: integer (nullable = true)
 |-- O2Satq: integer (nullable = true)
 |-- ChlorA: double (nullable = true)
 |-- Chlqua: integer (nullable = true)
 |-- Phaeop: double (nullable = true)
 |-- Phaqua: in

In [5]:
# See a snippet of what this dataframe looks like
bottle.show(2)

+-------+-------+-----------+--------------------+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+-----+------+-------+------+------+----+----+----+----+----+----+-------------------+
|Cst_Cnt|Btl_Cnt|     Sta_ID|            Depth_ID|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|BtlNum|RecInd|T_prec|T_qual|S_prec|S_qual|P_qual|O_qual|SThtaq|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3uM|NH3q|C14As1|C14A1p|C14A1q|C14As2|C14A2p|C14A2q|DarkAs|DarkAp|DarkAq|MeanAs|MeanAp|MeanAq|IncTim|LightP|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_NH4|R_CH

#### Remove columns that are not needed

Removing the four string columns because they aren't useful for our purposes
* Sta_ID: Line and Station
* Depth_ID: Uses the Cast_ID prefix ([Century]-[Year][Month][ShipCode]-[CastType][Julian Day]-[CastTime]-[Line][Sta]) but adds three additional variables: [Depth][Bottle]-[Rec_Ind]
* IncTim: Elapsed incubation time of the primary productivity experiment
* DIC Quality Comment: Quality Comment

Also removing the Cast and Bottle counts, which are essentially indexes (identifiers)
* 'Cst_Cnt': Auto-numbered Cast Count - all casts consecutively numbered. 1 is first station done
* 'Btl_Cnt': Auto-numbered Bottle count- all bottles ever sampled, consecutively numbered
* 'BtlNum': Bottle Number

In [6]:
# Dropping unneeded columns and viewing two rows of the resulting dataframe
deleteList1 = ["Sta_ID","Depth_ID","IncTim","DIC Quality Comment","Cst_Cnt","Btl_Cnt","BtlNum"]
bottle = bottle.drop(*deleteList1)

print("There are now", len(bottle.columns), "columns and", bottle.count(), "rows")
bottle.show(2, truncate=False)

There are now 67 columns and 864863 rows
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+-----+----+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+-----+------+-------+------+------+----+----+----+----+----+----+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|T_qual|S_prec|S_qual|P_qual|O_qual|SThtaq|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3uM|NH3q|C14As1|C14A1p|C14A1q|C14As2|C14A2p|C14A2q|DarkAs|DarkAp|DarkAq|MeanAs|MeanAp|MeanAq|LightP|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_NH4|R_CHLA|R_PHAEO|R_PRES|R_SAMP|DIC1|DIC2|TA1 |TA2 |pH2 |pH1 |
+------+------+------+------+------+-----+-------

#### Handle quality values

These columns indicate the quality codes because we're using the quantity measurements instead
* T_qual: Temperature Quality Code
* S_qual: Salinity Quality Code
* P_qual: Pressure Quality Code
* O_qual: Oxygen Quality Code
* 'O2Satq': Oxygen Saturation Quality Code
* 'Chlqua': Chlorophyll-a Quality Code
* 'Phaeop': Phaeophytin Quality Code
* 'Phaqua': Phosphate Quality Code
* 'PO4uM': Salinity Quality Code
* 'PO4q': Phosphate Quality Code
* 'SiO3qu': Quality Code
* 'NO2q': Quality Code
* 'NO3q': Nitrate Quality Code
* 'NH3q': Ammonium Quality Code
* 'C14A1q': 14C As1 Quality Code
* 'C14A2q': 14C As2 Quality Code
* 'DarkAq': 14C Assimilation Dark Bottle Quality Code
* 'MeanAq': Mean 14C Assimilation Quality Code


Before we delete these columns, we'll use their measurements (aka 8 because 8 means it's a bad recording of data) to delete bad quality data and THEN delete these columns.

//nts

In [7]:
# create the list of quality columns, but don't delete yet

#### Remove columns with low data count

We start by counting the number of NaNs and nulls in each column and reporting them in a new dataframe. Then the columns with less than 200,000 non-nulls are deleted.

In [8]:
# Counting the number of null/NaN rows per column and outputting that in a new dataframe

def getNullCounts(df):
    return df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])

nullCounter = getNullCounts(bottle)
nullCounter.show()

+------+------+------+------+------+------+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+------+--------+----------+-------+-----+-------+------+-------+------+------+------+------+------+------+-------+------+------+------+------+------+------+------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta| O2Sat|Oxy_µmol/Kg|RecInd|T_prec|T_qual|S_prec|S_qual|P_qual|O_qual|SThtaq|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua| PO4uM|  PO4q|SiO3uM|SiO3qu| NO2uM|  NO2q| NO3uM|  NO3q| NH3uM| NH3q|C14As1|C14A1p|C14A1q|C14As2|C14A2p|C14A2q|DarkAs|DarkAp|DarkAq|MeanAs|MeanAp|MeanAq|LightP|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|  R_O2|R_O2Sat|R_SIO3| R_PO4| R_NO3| R_NO2| R_NH4|R_CHLA|R_PHAEO|R_PRES|R_SAMP|  DIC1|  DIC2|   TA1|   TA2|   pH2|   pH1|
+------+------+------+------+-

In [9]:
# Deleting columns with less than 200000 non-nulls
thresh = maxRows - 200000
deleteList3 = []
for value in nullCounter.columns:
    if nullCounter.filter(nullCounter[value] > thresh).select(nullCounter[value]).collect():
        deleteList3.append(value)
bottle = bottle.drop(*deleteList3)

print("There are now", len(bottle.columns), "columns and", bottle.count(), "rows")
bottle.show(2, truncate=False)

There are now 45 columns and 864863 rows
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|S_prec|P_qual|O2Satq|ChlorA|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|0     |10.5  

#### Remove null values from chlorophyll column

Because this is the target column, we can only use the non-null rows. We also get rid of the duplicate chlorophyll column `ChlorA`, keeping `R_CHLA` because it has less non-null rows (though only by a few).

In [10]:
# Illustrating that we have two target columns that are essentially duplicates of each other
bottle.corr("ChlorA","R_CHLA")

0.9999995087108816

Because we only need one of these columns, we can delete the other. Then we drop all null rows of the R_CHLA column since it's our target.

In [11]:
# Looking at which of the two target columns have more NaNs in order to select which to delete
nullCounter.select(*["ChlorA","R_CHLA"]).show()

+------+------+
|ChlorA|R_CHLA|
+------+------+
|639591|639587|
+------+------+



In [12]:
# Delete the ChlorA column and drop all NaN rows in the R_CHLA column
bottle = bottle.drop("ChlorA")
bottle = bottle.dropna(subset="R_CHLA")

print("There are now", len(bottle.columns), "columns and", bottle.count(), "rows")
bottle.show(2, truncate=False)

There are now 44 columns and 225276 rows
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|S_prec|P_qual|O2Satq|Chlqua|Phaeop|Phaqua|PO4uM|PO4q|SiO3uM|SiO3qu|NO2uM|NO2q|NO3uM|NO3q|NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+----+------+------+-----+----+-----+----+----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|0     |19.23 |34.491|5.46  |24.575

#### Remove "duplicate" columns

As we can see from the column above, there are some columns that are essentially duplicates of each other but in different units of measurement. Of the remaining columns, here are their descriptions that will help us determine which features to compare for potential deletion:

In [14]:
# See what columns we still have
print(bottle.columns)

#nts: list this without the quality columns

['Depthm', 'T_degC', 'Salnty', 'O2ml_L', 'STheta', 'O2Sat', 'Oxy_µmol/Kg', 'RecInd', 'T_prec', 'S_prec', 'P_qual', 'O2Satq', 'Chlqua', 'Phaeop', 'Phaqua', 'PO4uM', 'PO4q', 'SiO3uM', 'SiO3qu', 'NO2uM', 'NO2q', 'NO3uM', 'NO3q', 'NH3q', 'C14A1q', 'C14A2q', 'DarkAq', 'MeanAq', 'R_Depth', 'R_TEMP', 'R_POTEMP', 'R_SALINITY', 'R_SIGMA', 'R_SVA', 'R_DYNHT', 'R_O2', 'R_O2Sat', 'R_SIO3', 'R_PO4', 'R_NO3', 'R_NO2', 'R_CHLA', 'R_PHAEO', 'R_PRES']


Chlorophyll
* 'ChlorA': Acetone extracted chlorophyll-a measured fluorometrically
* 'R_CHLA': Reported Chlorophyll-a (micrograms per liter)

Depth
* 'Depthm': Depth in meters
* 'R_Depth': Reported Depth (from pressure) in meters

Water density
* 'STheta': Potential Density of Water
* 'R_SIGMA': Reported Potential Density of water

Silicate
* 'SiO3uM': Micromoles Silicate per liter of seawater
* 'R_SIO3': Reported Silicate Concentration

Nitrite
* 'NO2uM': Micromoles Nitrite per liter of seawater
* 'R_NO2': Reported Nitrite Concentration

Nitrate
* 'NO3uM': Micromoles Nitrate per liter of seawater
* 'R_NO3': Reported Nitrate Concentration

Salinity
* 'Salnty': Practical Salinity Scale, 1978 (UNESCO, 1981a); Salinity of water
* 'R_SALINITY': Reported Salinity (from Specific Volume Anomoly, M³/Kg)

O2 saturation
* 'O2Sat': Percent Saturation; Oxygen Saturation
* 'R_O2Sat': Percent	Reported Oxygen Saturation

Oxygen
* 'O2ml_L': Oxygen in mL/L; Milliliters of dissolved oxygen per Liter seawater
* 'Oxy_µmol/Kg': Oxygen in micro moles per kilogram of seawater
* 'R_O2': Reported milliliters of oxygen per liter of seawater

Temperature
* 'T_degC': Temperature of Water
* 'R_TEMP': Reported Temperature (Celsius)
* 'R_POTEMP': Reported Potential Temperature (Celsius)

Phaeophytin
* 'Phaeop': Micrograms Phaeophytin per liter of seawater
* 'R_PHAEO': Reported Phaeophytin

//nts^^

Other
* 'S_prec': Salinity Units of Precision
* 'T_prec': Temperature Units of Precision
* 'RecInd': Record Indicator
* 'R_SVA': Reported Specific Volume Anomaly
* 'R_DYNHT': Reported Dynamic Height
* 'R_PO4': Reported Phosphate Concentration
* 'R_PRES': Pressure in decibars

In [15]:
# Setting up a function that will streamline the comparing process
nullCounter = getNullCounts(bottle)

def psudoDuplicateCheck(features):
    nullCounter.select(*features).show()

    if len(features)==2:
        print("correlation:", bottle.corr(features[0],features[1]))
    elif len(features)==3:
        print("1&2 correlation:", bottle.corr(features[0],features[1]))
        print("2&3 correlation:", bottle.corr(features[1],features[2]))
        print("1&3 correlation:", bottle.corr(features[0],features[2]))

In [16]:
nullCounter.show()

+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+------+------+------+-----+------+-----+------+-----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|Depthm|T_degC|Salnty|O2ml_L|STheta|O2Sat|Oxy_µmol/Kg|RecInd|T_prec|S_prec|P_qual|O2Satq|Chlqua|Phaeop|Phaqua|PO4uM|  PO4q|SiO3uM|SiO3qu|NO2uM|  NO2q|NO3uM|  NO3q| NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_POTEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+-----+-----------+------+------+------+------+------+------+------+------+-----+------+------+------+-----+------+-----+------+-----+------+------+------+------+-------+------+--------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|     0|  3088|  3433|  3610|  3781| 4500|       4502| 

In [17]:
Depths = ["Depthm","R_Depth"]
print("Depths NaN count:")
psudoDuplicateCheck(Depths)

WD = ["STheta","R_SIGMA"]
print("\nWater density NaN count:")
psudoDuplicateCheck(WD)

Silicate = ["SiO3uM","R_SIO3"]
print("\nSilicate NaN count:")
psudoDuplicateCheck(Silicate)

Nitrite = ["NO2uM","R_NO2"]
print("\nNitrite NaN count:")
psudoDuplicateCheck(Nitrite)

Nitrate = ["NO3uM","R_NO3"]
print("\nNitrate NaN count:")
psudoDuplicateCheck(Nitrate)

Salinity = ["Salnty","R_SALINITY"]
print("\nSalinity NaN count:")
psudoDuplicateCheck(Salinity)

Saturation = ["O2Sat","R_O2Sat"]
print("\nO2 Saturation NaN count:")
psudoDuplicateCheck(Saturation)

Oxygen = ["O2ml_L","Oxy_µmol/Kg","R_O2"]
print("\nOxygen NaN count:")
psudoDuplicateCheck(Oxygen)

Temperature = ["T_degC","R_TEMP","R_POTEMP"]
print("\nTemperature NaN count:")
psudoDuplicateCheck(Temperature)

Phaeophytin = ['Phaeop', 'R_PHAEO']
print("\nPhaeophytin NaN count:")
psudoDuplicateCheck(Phaeophytin)

Depths NaN count:
+------+-------+
|Depthm|R_Depth|
+------+-------+
|     0|      0|
+------+-------+

correlation: 0.9999999949168986

Water density NaN count:
+------+-------+
|STheta|R_SIGMA|
+------+-------+
|  3781|   3655|
+------+-------+

correlation: 0.9775619506024495

Silicate NaN count:
+------+------+
|SiO3uM|R_SIO3|
+------+------+
|  8947|  8941|
+------+------+

correlation: 0.9999991048864016

Nitrite NaN count:
+-----+-----+
|NO2uM|R_NO2|
+-----+-----+
|14951|14945|
+-----+-----+

correlation: 0.9999753963583337

Nitrate NaN count:
+-----+-----+
|NO3uM|R_NO3|
+-----+-----+
| 9573| 9567|
+-----+-----+

correlation: 0.9999998732099501

Salinity NaN count:
+------+----------+
|Salnty|R_SALINITY|
+------+----------+
|  3433|      3433|
+------+----------+

correlation: 0.9999999893049997

O2 Saturation NaN count:
+-----+-------+
|O2Sat|R_O2Sat|
+-----+-------+
| 4500|   4352|
+-----+-------+

correlation: 0.9903641742840068

Oxygen NaN count:
+------+-----------+----+
|O

We now delete the columns with fewer null if they have a correlation coeffecients of 0.97 or higher.

In the case of a tie, we can refer to the pattern that is very apparent in no-tie cases, which is that the "reported" (columns starting with `R_`) have the higher non-null count. Therefore if two columns have equal null counts and high enough correlation, we delete the not-"reported" column.

In [18]:
# Deleting columns that are duplicates of some other column
deleteList4 = ["Depthm","STheta","SiO3uM","NO2uM","NO3uM","Salnty","O2Sat",\
               "O2ml_L","Oxy_µmol/Kg","T_degC","R_POTEMP", "Phaeop"]
bottle = bottle.drop(*deleteList4)

print("There are now", len(bottle.columns), "columns and", bottle.count(), "rows")
bottle.show(2, truncate=False)

There are now 32 columns and 225276 rows
+------+------+------+------+------+------+------+-----+----+------+----+----+----+------+------+------+------+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|RecInd|T_prec|S_prec|P_qual|O2Satq|Chlqua|Phaqua|PO4uM|PO4q|SiO3qu|NO2q|NO3q|NH3q|C14A1q|C14A2q|DarkAq|MeanAq|R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------+------+------+------+-----+----+------+----+----+----+------+------+------+------+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|3     |2     |3     |9     |null  |null  |null  |null |9   |9     |9   |null|9   |9     |9     |9     |9     |0.0    |19.23 |34.491    |24.57  |335.3|0.0    |5.46|103.9  |null  |null |1.3  |null |0.64  |0.47   |0     |
|3     |2     |3     |9     |null  |null  |null  |null |9   |9     |9   |null|9

In [19]:
#nts: organize thoughts

# Create temp dataframe and remove Quality Code columns from temp dataframe for lasso
deleteList2 = ['T_qual','S_qual','P_qual','O_qual','O2Satq','Chlqua', 'Phaqua','PO4q','SiO3qu','NO2q','NO3q','NH3q','C14A1q','C14A2q','DarkAq','MeanAq']

bottleTest = bottle.alias('bottleTest')
bottleTest = bottleTest.drop(*deleteList2)

#### Fill null values

In [20]:
# Using the function from PA3
from pyspark.ml.feature import Imputer

def fill_na(df, strategy):    
    imputer = Imputer(
        strategy=strategy,
        inputCols=df.columns, 
        outputCols=["{}_imputed".format(c) for c in df.columns]
    )
    
    new_df = imputer.fit(df).transform(df)
    
    ''' Select the newly created columns with all filled values '''
    new_df = new_df.select([c for c in new_df.columns if "imputed" in c])
    
    for col in new_df.columns:
        new_df = new_df.withColumnRenamed(col, col.split("_imputed")[0])
        
    return new_df

In [21]:
# Filling in the remaining null rows with the mean of the column and saving this newly filled in dataframe as a new variable

bottleTest = fill_na(bottleTest, 'mean')

print("There are still", len(bottleTest.columns), "columns and", bottleTest.count(), "rows")
bottleTest.show(2, truncate=False)

There are still 19 columns and 225276 rows
+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+------+-------+------+
|RecInd|T_prec|S_prec|PO4uM             |R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3            |R_PO4             |R_NO3|R_NO2              |R_CHLA|R_PHAEO|R_PRES|
+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+------+-------+------+
|3     |2     |3     |0.9506321528926772|0.0    |19.23 |34.491    |24.57  |335.3|0.0    |5.46|103.9  |11.529812096979274|0.9506145966112527|1.3  |0.05539378408325813|0.64  |0.47   |0     |
|3     |2     |3     |0.9506321528926772|10.0   |19.22 |34.492    |24.57  |335.3|0.03   |5.46|103.9  |11.529812096979274|0.9506145966112527|1.9  |0.05539378408325813|0.66  |0.38   |10    |
+------+----

In [22]:
# Double checking that all null values are filled
getNullCounts(bottleTest).show()

+------+------+------+-----+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|RecInd|T_prec|S_prec|PO4uM|R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3|R_PO4|R_NO3|R_NO2|R_CHLA|R_PHAEO|R_PRES|
+------+------+------+-----+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+
|     0|     0|     0|    0|      0|     0|         0|      0|    0|      0|   0|      0|     0|    0|    0|    0|     0|      0|     0|
+------+------+------+-----+-------+------+----------+-------+-----+-------+----+-------+------+-----+-----+-----+------+-------+------+



#### Lasso regression

To better estimate what columns are most important to our model, we use a lasso regression. Because spark LassoWithSGD has depricated, we use LinearRegression with `elasticNetParam = 1.0` for Lasso regression equivalent.

In order to use this, `label` and `features` columns need to be specified. We rename `R_CHLA` to `label` and create a features column with VectorAssembler.

In [23]:
# Rename the target chlorophyll column to 'label'
bottleTest = bottleTest.withColumnRenamed('R_CHLA', 'label')

In [24]:
# Creating 'features' column
from pyspark.ml.feature import VectorAssembler

# (interm step) make list of column names other than 'label,' AKA make a list of the features
features = bottleTest.columns
features.remove('label')

bottleTest = VectorAssembler(outputCol="features").setInputCols(features).transform(bottleTest)

bottleTest.show(2, truncate=False)

+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+-----+-------+------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|RecInd|T_prec|S_prec|PO4uM             |R_Depth|R_TEMP|R_SALINITY|R_SIGMA|R_SVA|R_DYNHT|R_O2|R_O2Sat|R_SIO3            |R_PO4             |R_NO3|R_NO2              |label|R_PHAEO|R_PRES|features                                                                                                                                              |
+------+------+------+------------------+-------+------+----------+-------+-----+-------+----+-------+------------------+------------------+-----+-------------------+-----+-------+------+---------------------------------------------------------------------------------------------------------------------------------------

In [25]:
# Applying lasso regression where regParam= 0.0025 in place of alpha
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(elasticNetParam = 1.0, regParam=0.0025, solver="normal", maxIter=1000, standardization=True)
model = lr.fit(bottleTest)

2022-05-25 04:09:33,925 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
2022-05-25 04:09:33,932 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


##### using the coeff list

//nts: here are the "most useful" features. i'm not sure if i should be deleting the "useless" ones or just dropping nulls from the other columns

In [26]:
# Find the "relevant" and "not useful" features, as according to the lasso regresion when regParam=0.0025

coeff = model.coefficients
usefulFeatures = []
deleteList5 = []

print("(For-loop of", len(coeff), "iterations)")
for i in range(len(coeff)):
    if coeff[i] != 0:
        usefulFeatures.append(features[i])
    else:
        deleteList5.append(features[i])

print("When regParam="+str(lr.getRegParam())+", this lasso model indicates the most useful columns to predicting chlorophyll are:", usefulFeatures)

(For-loop of 18 iterations)
When regParam=0.0025, this lasso model indicates the most useful columns to predicting chlorophyll are: ['S_prec', 'R_TEMP', 'R_SALINITY', 'R_DYNHT', 'R_O2', 'R_O2Sat', 'R_SIO3', 'R_NO3', 'R_NO2', 'R_PHAEO']


In [27]:
# Moving back to dataframe before nulls were filled with means, drop the null rows of the "most useful" features
bottle = bottle.dropna(subset=usefulFeatures)

print("There are still", len(bottle.columns), "columns and", bottle.count(), "rows")

There are still 32 columns and 208035 rows


In [28]:
# Moving back to dataframe before nulls were filled with means, drop the "not useful" features
bottle = bottle.drop(*deleteList5)

print("There are still", len(bottleTest.columns), "columns and", bottleTest.count(), "rows")

There are still 20 columns and 225276 rows


In [29]:
# Now that we're done with this temporary dataframe, we can delete it
del bottleTest

In [30]:
# Creating a map/2D-list of the quality columns with the measurement columns they correspond to
# in order to delete the low-quality records from our current bottle dataframe
qualityColWithRCol = [deleteList2,
                      ['R_TEMP', 'R_SALINITY', 'R_PRES', 'R_O2', 'R_O2Sat', 'R_CHLA', 'Phaeop', 'R_PO4' \
                      'R_SIO3', 'R_NO2', 'R_NO3', 'R_Nuts', 'C14As1', 'C14As2', 'DarkAp', 'MeanAs']]

In [31]:
# Print out the number of bad-quality recordings per column in the bottle dataframe
for i in range(len(qualityColWithRCol[0])):
    colName = qualityColWithRCol[0][i]
    if(colName in bottle.columns):
        numBadData = bottle.filter(bottle[qualityColWithRCol[0][i]] == 8).count()
        print(f"Number of bad quality data for {i}: {numBadData}")
    
# Dropping bad quality rows of quality columns
for i in range(len(qualityColWithRCol[0])):
    colName = qualityColWithRCol[0][i]
    if(colName in bottle.columns):
        bottle = bottle.filter((bottle[qualityColWithRCol[0][i]] != 8) | \
                               (bottle[qualityColWithRCol[0][i]].isNull()))

Number of bad quality data for 2: 0
Number of bad quality data for 4: 25
Number of bad quality data for 5: 88
Number of bad quality data for 6: 91
Number of bad quality data for 7: 64
Number of bad quality data for 8: 47
Number of bad quality data for 9: 1295
Number of bad quality data for 10: 2041
Number of bad quality data for 11: 0
Number of bad quality data for 12: 7
Number of bad quality data for 13: 7
Number of bad quality data for 14: 7
Number of bad quality data for 15: 7


In [32]:
# Now we can get rid of the quality columns since they've served their purpose
bottle = bottle.drop(*deleteList2)

#### Feature correlation

Finding columns that are similar and analyse if we can get rid of null columns without removal of variance.

In [33]:
from pyspark.sql.types import StructType, StructField, IntegerType

In [34]:
columns = ['col1', 'col2', 'correlation']
schema = StructType([
  StructField('col1', StringType(), False),
  StructField('col2', StringType(), False),
  StructField('correlation', IntegerType(), False)
  ])
highCorrs = spark.createDataFrame(spark.sparkContext.emptyRDD() ,schema)
rangeColumns = range(len(bottle.columns))
for i in rangeColumns:
    for j in rangeColumns:
        if(i != j):
            correlation = bottle.corr(bottle.columns[i], bottle.columns[j])
            if(correlation > .9):
                newCorr = spark.createDataFrame([(bottle.columns[i], bottle.columns[j], correlation)], columns)
                highCorrs = highCorrs.union(newCorr)
            else: 
                 continue;

In [35]:
highCorrs.show()

+-------+-------+------------------+
|   col1|   col2|       correlation|
+-------+-------+------------------+
|   R_O2|R_O2Sat|0.9881367985673449|
|R_O2Sat|   R_O2|0.9881367985673449|
| R_SIO3|  R_NO3|0.9754158165712739|
|  R_NO3| R_SIO3|0.9754158165712741|
+-------+-------+------------------+



Because R_PO4 has 998 nulls and is highly correlated with R_NO3 and R_SIO3, we will drop this column with little reduction in variance. R_SVA also contains nulls and is highly correlated with R_Temp, so we will drop this column as well. 

nts

In [36]:
bottle = bottle.drop('R_O2')
bottle.withColumnRenamed('R_O2Sat', 'R_O2, R_O2Sat')
bottle = bottle.drop('R_SIO3')
bottle.withColumnRenamed('R_NO3', 'R_SIO3, R_NO3')
bottle.show(2)

+------+------+----------+-------+-------+-----+-----+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2Sat|R_NO3|R_NO2|R_PHAEO|
+------+------+----------+-------+-------+-----+-----+-------+
|     3| 15.34|    33.492|    0.0|  102.8|  0.1|  0.0|   0.19|
|     3| 15.34|    33.492|  0.003|  102.8|  0.1|  0.0|   0.19|
+------+------+----------+-------+-------+-----+-----+-------+
only showing top 2 rows



#### Make sure column values are the right type

In [37]:
# Check the dtypes of each column
bottle.printSchema()

root
 |-- S_prec: integer (nullable = true)
 |-- R_TEMP: double (nullable = true)
 |-- R_SALINITY: double (nullable = true)
 |-- R_DYNHT: double (nullable = true)
 |-- R_O2Sat: double (nullable = true)
 |-- R_NO3: double (nullable = true)
 |-- R_NO2: double (nullable = true)
 |-- R_PHAEO: double (nullable = true)



In [38]:
# Row count, and comparison to original row count
print("There are now", len(bottle.columns), "columns and", bottle.count(), "rows (which is", maxRows-bottle.count(), "less than the original row count)")

There are now 8 columns and 205188 rows (which is 659675 less than the original row count)


#### Summary Statistics and graphs

In [39]:
# See the detailed numerical description of the current dataframe
bottle.describe().show(vertical=True)

-RECORD 0--------------------------
 summary    | count                
 S_prec     | 205188               
 R_TEMP     | 205188               
 R_SALINITY | 205188               
 R_DYNHT    | 205188               
 R_O2Sat    | 205188               
 R_NO3      | 205188               
 R_NO2      | 205188               
 R_PHAEO    | 205188               
-RECORD 1--------------------------
 summary    | mean                 
 S_prec     | 2.999376181842993    
 R_TEMP     | 12.89435291537499    
 R_SALINITY | 33.50207719262336    
 R_DYNHT    | 0.21029396455934762  
 R_O2Sat    | 81.11208355264498    
 R_NO3      | 9.625738834631228    
 R_NO2      | 0.0561527477241122   
 R_PHAEO    | 0.18895593309549663  
-RECORD 2--------------------------
 summary    | stddev               
 S_prec     | 0.024968621232654243 
 R_TEMP     | 3.0295041279649153   
 R_SALINITY | 0.30430755583854363  
 R_DYNHT    | 0.15528817699558983  
 R_O2Sat    | 25.23765304716695    
 R_NO3      | 10.29372710774

In [40]:
getNullCounts(bottle).show()

+------+------+----------+-------+-------+-----+-----+-------+
|S_prec|R_TEMP|R_SALINITY|R_DYNHT|R_O2Sat|R_NO3|R_NO2|R_PHAEO|
+------+------+----------+-------+-------+-----+-----+-------+
|     0|     0|         0|      0|      0|    0|    0|      0|
+------+------+----------+-------+-------+-----+-----+-------+



## Feature Selection and Target Cleaning

#### Distribution of data for cholorophyll levels, summary stats

In [41]:
bottle.agg(min('R_CHLA'), max('R_CHLA'), mean('R_CHLA'), stddev('R_CHLA'), count('R_CHLA'), skewness('R_CHLA')).show()

AnalysisException: cannot resolve 'R_CHLA' given input columns: [R_DYNHT, R_NO2, R_NO3, R_O2Sat, R_PHAEO, R_SALINITY, R_TEMP, S_prec];
'Aggregate [min('R_CHLA) AS min(R_CHLA)#409911, max('R_CHLA) AS max(R_CHLA)#409912, avg('R_CHLA) AS avg(R_CHLA)#409913, stddev_samp('R_CHLA) AS stddev_samp(R_CHLA)#409922, count('R_CHLA) AS count(R_CHLA)#409923, skewness('R_CHLA) AS skewness(R_CHLA)#409933]
+- Project [S_prec#31, R_TEMP#66, R_SALINITY#68, R_DYNHT#71, R_O2Sat#73, R_NO3#76, R_NO2#77, R_PHAEO#80]
   +- Project [S_prec#31, R_TEMP#66, R_SALINITY#68, R_DYNHT#71, R_O2Sat#73, R_NO3#76, R_NO2#77, R_CHLA#79, R_PHAEO#80]
      +- Project [S_prec#31, R_TEMP#66, R_SALINITY#68, R_DYNHT#71, R_O2Sat#73, R_SIO3#74, R_NO3#76, R_NO2#77, R_CHLA#79, R_PHAEO#80]
         +- Project [S_prec#31, R_TEMP#66, R_SALINITY#68, R_DYNHT#71, R_O2#72, R_O2Sat#73, R_SIO3#74, R_NO3#76, R_NO2#77, R_CHLA#79, R_PHAEO#80]
            +- Filter (NOT (MeanAq#62 = 8) OR isnull(MeanAq#62))
               +- Filter (NOT (DarkAq#59

Looking at this summary data of our chlorophyll levels, the data is highly positively skewed. This is displayed in our skewness value but also the difference between the maximum value and the mean.

#### Define cholorophyll category levels: low, medium, high?

As displayed in Figure 4 of this article,https://www.nature.com/scitable/knowledge/library/the-biological-productivity-of-the-ocean-70631104/, the chlorophyll concentrations in the ocean can be split into three category levels, low, medium, and high. Our low category range will be between 0 > x > .1ug/l, medium range will be .1 >= x > 1ug/l, and high concentrations will be x >= 1ug/l.

#### Map cholorophyll levels to defined categories and output graphs/summary stats and Add target column with category label to dataframe

In [ ]:
lowThreshold = .1
medThreshold = 1

In [ ]:
bottle = bottle.withColumn(
    'Target',
     when((col("R_CHLA") < lowThreshold), 0)\
    .when((col("R_CHLA").between(lowThreshold, medThreshold)), 1)\
    .when((col("R_CHLA") > medThreshold), 2)\
    .otherwise(10)
)

In [ ]:
bottle.groupby('Target').count().show()

## Models

#### Split data into 80/10/10 train, test, validation set

In [ ]:
#Checking schema of data again. Using bottleTest df instead of 'bottle' because it has no null values
#and used vector assembler to create feature vector.
bottle.printSchema()

In [ ]:
features = bottle.columns
features.remove('Target')
bottle = VectorAssembler(outputCol="features_unscaled").setInputCols(features).transform(bottle)

In [ ]:
bottle.show(2, truncate=False)

In [ ]:
seed = 42
train, test, validation = bottle.randomSplit([0.80, 0.10, 0.10], seed=seed)
print('Train dataset count:', train.count())
print('Test dataset count:', test.count())
print('Validation dataset count:', validation.count())

#### Scale Data

In [ ]:
from pyspark.ml.feature import MinMaxScaler

mmScaler = MinMaxScaler(inputCol='features_unscaled', outputCol='features')
mmModel = mmScaler.fit(train)

In [ ]:
trainscaled = mmModel.transform(train)
testscaled = mmModel.transform(test)
valscaled = mmModel.transform(validation)

#### Check for Imblance in datasets

In [ ]:
trainscaled.groupby('target').count().show()

In [ ]:
testscaled.groupby('target').count().show()

In [ ]:
valscaled.groupby('target').count().show()

The datasets are well balanced with proportional amounts of each feature in each set.

### Logistic Regresssion

See what models use the train data to predict labels, train and validation accuracy scores

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
regParams = [0.001, 0.002, 0.003, 0.004]
elasticNetParams = [0.5, 0.6, 0.7]

for r in regParams:
    for e in elasticNetParams:
        lr = LogisticRegression(featuresCol='features', labelCol='Target', predictionCol='prediction', maxIter=10, regParam=r, elasticNetParam=e)
        lrModel = lr.fit(trainscaled)
        print('regParam', r, 'elasticNetParam', e, 'accuracy', lrModel.summary.accuracy)

In [ ]:
lr = LogisticRegression(featuresCol='features', labelCol='Target', predictionCol='prediction', \
                        maxIter=10, regParam=0.001, elasticNetParam=0.6)
lrModel = lr.fit(trainscaled)

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))
    
print('-------------------------------------------')

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFalse Positive Rate: %s\nTrue Positive Rate: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

In [ ]:
predictVal = lrModel.evaluate(valscaled)
accuracy = predictVal.accuracy
falsePositiveRate = predictVal.weightedFalsePositiveRate
truePositiveRate = predictVal.weightedTruePositiveRate
fMeasure = predictVal.weightedFMeasure()
precision = predictVal.weightedPrecision
recall = predictVal.weightedRecall

print("Accuracy: %s\nFalse Positive Rate: %s\nTrue Positive Rate: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

In [ ]:
import numpy as np
coeff = lrModel.coefficientMatrix.toArray()
coeff = np.argmax(coeff, axis=1)

In [ ]:
for i in range(len(coeff)):
    print("For label", i, "the most important column is", valscaled.columns[coeff[i]])

### Decision Tree

Can we increase accuracy from previous steps? Can we figure out which features matter the most?

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(labelCol="Target", seed=seed)
dtModel = dt.fit(trainscaled)
dtModel.setFeaturesCol("features")
trainResult = dtModel.transform(trainscaled)
valResult =  dtModel.transform(valscaled)
print(trainResult.columns)

In [ ]:
def calculate_accuracy(df):
    correct = df.filter(df["Target"] == df["prediction"]).count()
    incorrect = df.filter(df["Target"] != df["prediction"]).count()
    totalRows = df.count()
    accuracy = correct/totalRows
    error = incorrect/totalRows
    print('Accuracy: ', accuracy)
    print('Error Rate: ', error)

#### Train and Test Accuracy Scores

In [ ]:
calculate_accuracy(trainResult)
calculate_accuracy(valResult)

In [ ]:
print("Model Depth:", dtModel.depth)
print("Model Number of Nodes:", dtModel.numNodes)

In [ ]:
print('feature importance', dtModel.featureImportances)

In [ ]:
dtModel.featureImportances.toArray()

In [ ]:
print(testscaled.columns)

# Stop spark session

In [ ]:
#spark.stop()